In [2]:
import sys
sys.path.insert(0, "/Users/rstiskalek/Projects/candel")

import numpyro
numpyro.set_host_device_count(12)

import numpy as np
import matplotlib.pyplot as plt
import scienceplots

from scipy.stats import norm
import candel

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
config_path = "/Users/rstiskalek/Projects/CANDEL/scripts/runs/config_shoes.toml"
data = candel.pvdata.load_SH0ES_from_config(config_path, )

model = candel.model.SH0ESModel(config_path, data)


print("Priors are:")
print(model.prior_dist_name)

2025-07-17 22:20:57 warning: zero width found in the priors. Setting it to 1e-5.
2025-07-17 22:20:57 indices of zero width: [44]
2025-07-17 22:20:57 found zero-width prior at index 44. Setting it to 0.
2025-07-17 22:20:57 normalizing the Manticore LOS density.
2025-07-17 22:20:57 Masking Cepheids with cz_cmb > 3300 km/s: Keeping 35 out of 37.
2025-07-17 22:20:57 replacing prior of `M_B` with a delta function.
2025-07-17 22:20:57 replacing prior of `A_covmat` with a delta function.
2025-07-17 22:20:57 loading host galaxy LOS interpolators.
2025-07-17 22:20:57 Popped the following keys with `None` values from data: czcmb_SN_HF, e_czcmb_SN_HF, RA_SN_HF, dec_SN_HF
2025-07-17 22:20:57 Converting host RA/dec to Cartesian coordinates.
2025-07-17 22:20:57 set the following attributes: mag_cepheid, logP, OH, C_Cepheid, L_Cepheid, L_Cepheid_host_dist, Cepheids_only, num_cepheids, num_hosts, Y_SN_Cepheid, C_SN_Cepheid, L_SN_Cepheid, L_SN_Cepheid_dist, mag_SN_unique_Cepheid_host, C_SN_unique_Cephe

In [ ]:
samples = candel.run_SH0ES_inference(model, )

2025-07-17 22:20:58 running inference on devices: cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu), cpu(cpu)
2025-07-17 22:20:58 using NumPyro platform: CPU


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

...

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.7),)

# Histogram of H0 samples (normalize to max height = 1)
counts, bins, _ = ax.hist(
    samples["H0"], bins="auto", density=True, alpha=0.9, color="0.3",
    label=r"$H_0$ samples"
)
max_density = counts.max()

# Define a range for the Gaussians
H0_vals = np.linspace(60, 80, 1000)

# SH0ES (e.g. Riess+2021), normalized to match
H0_SH0ES = 73.04
H0_SH0ES_err = 1.04
shoes_pdf = norm.pdf(H0_vals, loc=H0_SH0ES, scale=H0_SH0ES_err)
# shoes_pdf /= shoes_pdf.max()  # normalize peak to 1
# shoes_pdf *= max_density      # scale to match sample histogram
ax.plot(H0_vals, shoes_pdf, color="tab:red", label="SH0ES")

# Planck (e.g. Planck 2018), normalized to match
H0_Planck = 67.4
H0_Planck_err = 0.5
planck_pdf = norm.pdf(H0_vals, loc=H0_Planck, scale=H0_Planck_err)
# planck_pdf /= planck_pdf.max()
# planck_pdf *= max_density
ax.plot(H0_vals, planck_pdf, color="tab:blue", label="Planck")

ax.set_xlabel(r"$H_0~[\mathrm{km/s/Mpc}]$")
ax.set_ylabel("Normalized density")
ax.legend()

plt.tight_layout()
fig.savefig("/Users/rstiskalek/Downloads/H0_samples.png", bbox_inches="tight", dpi=450)
plt.show()

In [ ]:
from scipy.stats import norm
import posterior_agreement


H0 = norm(69.26, 1.06).rvs(50_000)
# H0_test = norm(67.4, 0.5).rvs(50000)
H0_test = norm(73.04, 1.04).rvs(50000)



chains = (H0, H0_test)
agreement_1d = posterior_agreement.compute_agreement(chains)

print("p-value %.3f, corresponding to %.4f sigmas"%(agreement_1d.PTE, agreement_1d.sigma))